# Transformer

- Transformer는 **Self-Attention** 메커니즘을 활용해 입력 시퀀스 내 요소들 간의 관계를 동시 처리하는 딥러닝 모델

- 인코더-디코더 구조를 갖고 있으며, 각각 다층의 attention 및 feedforward 레이어로 구성

- 앞 단원에서는 순환 신경망을 거친 encoder output과 마찬가지로 순환 신경망을 거쳐 나온 디코더 아웃풋 간의 attention 구조를 살펴보았다.

- Transformer는 순환 신경망 과정을 생략하고 대신 self-attention 구조를 이용.

- 순차적인 처리 없이도 병렬 연산이 가능해, 번역·요약·질문응답 등 다양한 자연어 처리 작업에서 뛰어난 성능을 보임

- 과목 특성상 자연어 처리작업을 하기는 어려우므로 이 단원에서는 Transformer의 구조에 대해 간략히 알아본다.

## Positional encoding

Transformer 아키텍처는 기본적으로 순서에 의존하지 않는 구조인 self-attention 메커니즘을 기반으로 한다.   
    
추후 알아볼 self-attention은 입력 시퀀스를 처리할 때 각 위치의 값들이 서로 독립적으로 처리되므로, 순서 정보가 내재되어 있지 않다. 
- RNN과 같은 전통적인 시퀀스 모델들은 자연스럽게 순서를 처리하지만, Transformer는 병렬 처리가 가능하도록 설계되어 순서 정보가 내재되어 있지 않다.

따라서, 시퀀스의 순서 정보를 모델에 제공하기 위해 positional encoding을 도입하여, 입력 시퀀스의 각 위치에 대한 순서 정보를 학습할 수 있게 하였다.

### Positional Encoding 방식

Transformer에서 사용되는 대표적인 positional encoding 방식은 사인과 코사인 함수를 사용한다. 

각 차원에 대해 다른 주기를 가지는 사인과 코사인 값을 사용하여 각 위치의 인코딩을 생성한다.

이러한 방식은 시퀀스의 길이에 관계없이 위치 정보를 제공할 수 있고, 모델이 이를 통해 위치 정보를 학습할 수 있도록 돕는다.

Positional enconding에 사용되는 수식은 다음과 같다.

$$\textrm{PE}{(p, 2i)} = \sin\left(\frac{p}{N^{2i/d}} \right), \quad \textrm{PE}{(p, 2i+1)} = \cos\left(\frac{p}{N^{2i/d}}\right)$$

- $p$ : 시간축에 대한 인덱스
- $2i, 2i+1$ : 입력 시퀀스의 차원에 대한 인덱스, 각, 짝수와 홀수 인덱스를 의미
- $N$ : 큰 양수
- $d$ : 입력 시퀀스의 차원

`numpy`를 이용해 간단히 표현하면 다음과 같다.

In [1]:
import numpy as np
import math

# Define sequence length and model dimension
timestep = 6
dim = 4

pe = np.zeros((timestep, dim))
    
# Calculate the position indices for the sequence
position = np.arange(0, timestep)[:, np.newaxis]

N = 10000.0
# Calculate the dimension indices
div_term = 1 / np.power(N, np.arange(0, dim, 2) / dim)

# Apply the sin function to even indices and cos function to odd indices
pe[:, 0::2] = np.sin(position * div_term)  # Apply sin to even indices
pe[:, 1::2] = np.cos(position * div_term)  # Apply cos to odd indices

# Print the positional encoding matrix
print(pe)

[[ 0.          1.          0.          1.        ]
 [ 0.84147098  0.54030231  0.00999983  0.99995   ]
 [ 0.90929743 -0.41614684  0.01999867  0.99980001]
 [ 0.14112001 -0.9899925   0.0299955   0.99955003]
 [-0.7568025  -0.65364362  0.03998933  0.99920011]
 [-0.95892427  0.28366219  0.04997917  0.99875026]]


위에서 출력된 행렬의 행은 시간축이고, 열은 입력 시퀀스의 차원을 나타낸다.

같은 거리에 있는 positional encoding 벡터들을 내적하였을 때, 동일한 값을 가짐을 확인해 보자.

In [2]:
pe[0, :] @ pe[1, :], pe[1, :] @ pe[2, :], pe[2, :] @ pe[3, :], pe[3, :] @ pe[4, :], pe[4, :] @ pe[5, :]

(1.540252306284805,
 1.5402523062848048,
 1.540252306284805,
 1.540252306284805,
 1.540252306284805)

In [3]:
pe[0, :] @ pe[2, :], pe[1, :] @ pe[3, :], pe[2, :] @ pe[4, :], pe[3, :] @ pe[5, :]

(0.5836531701194354,
 0.5836531701194354,
 0.5836531701194354,
 0.5836531701194355)

Positional encoding을 담당하는 keras layer를 클래스로 정의하여 보자.

- `class PositionalEncoding(layers.Layer):` TensorFlow/Keras에서 사용자 정의 레이어를 만들기 위한 정의

  - `layers.Layer`를 상속했기 때문에 Keras의 커스텀 레이어가 됨
 
  - 이 레이어는 `__init__`, `call`, 등의 메서드를 통해 정의된 사용자 정의 동작을 수행할 수 있음

- 편의상, 이 코드는 짝수의 `dim`에 대해서만 작동하도록 만들었음

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class PositionalEncoding(layers.Layer):
    def __init__(self, max_len, dim):
        super(PositionalEncoding, self).__init__()
        pos_encoding = np.zeros((max_len, dim))
        positions = np.arange(0, max_len)[:, np.newaxis]

        N = 10000.0
        div_term =  1 / np.power(N, np.arange(0, dim, 2) / dim)
        
        pos_encoding[:, 0::2] = np.sin(positions * div_term)
        pos_encoding[:, 1::2] = np.cos(positions * div_term)
        
        pos_encoding = pos_encoding[np.newaxis, ...]
        self.pos_encoding = tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

잘 작동하는지 확인해 보자.

- shape이 `(1, timestep, dim)`인 제로 시퀀스 입력에 대해 위치 정보를 부여하는 positional encoding이 잘 작동하는지 테스트

- 아래 출력 결과를 보면, 출력이 timestep마다 다르며, 이는 positional encoding이 위치 정보를 명시적으로 부여했다는 의미 

In [5]:
PositionalEncoding(timestep, dim)(np.zeros((1, timestep, dim)))

<tf.Tensor: shape=(1, 6, 4), dtype=float32, numpy=
array([[[ 0.        ,  1.        ,  0.        ,  1.        ],
        [ 0.84147096,  0.5403023 ,  0.00999983,  0.99995   ],
        [ 0.9092974 , -0.41614684,  0.01999867,  0.9998    ],
        [ 0.14112   , -0.9899925 ,  0.0299955 ,  0.99955004],
        [-0.7568025 , -0.6536436 ,  0.03998933,  0.9992001 ],
        [-0.9589243 ,  0.2836622 ,  0.04997917,  0.99875027]]],
      dtype=float32)>

## Scaled dot-product attention

Attention은 이전 단원에서 살펴본 것처럼 query, key, value들의 dot product들로 계산한다.

Attention의 목적은 query와 key의 유사성을 dot-product로 계산하여, 이를 기반으로 value에 가중치를 부여하는 것이다.

* **Query (Q)**: 현재 우리가 “관심 있는 항목”에 대한 표현. 디코더의 현재 시점 상태 등이 사용됨.
  
* **Key (K)**: 인코더의 각 위치에 대한 설명. 디코더가 어떤 인코더 위치에 주목할지 판단할 기준.
  
* **Value (V)**: 실제로 정보를 얻고자 하는 대상. 보통 Key와 동일한 위치에서 생성된 벡터.
  

### 가중치 행렬 연산

Transformer의 Attention 메커니즘에서 Query, Key, Value는 각각 입력 시퀀스 `X_q`, `X_k`, `X_v`에 가중치 행렬 `W_q`, `W_k`, `W_v`를 곱하여 생성된다.

- 만약 쿼리 입력 시퀀스 `X_q`와 키 입력 시퀀스 `X_k`가 같다면 self-attention이라고 부른다. 

- 쿼리 입력 시퀀스 `X_q`와 값 입력 시퀀스 `X_v`는 같을 수도 있고 다를 수도 있다.

쿼리 입력 시퀀스의 차원을 `d_model_q`, 키 입력 시퀀스의 차원을 `d_model_q`이라고 하자. 

- 쿼리 입력 시퀀스의 shape : `(batch_size, seq_len_q, d_model_q)`

- 키 입력 시퀀스의 shape : `(batch_size, seq_len_k, d_model_k)`

가중치 행렬들의 shape은 다음과 같다.

* `W_q`: `(d_model_q, key_dim)`  
* `W_k`: `(d_model_k, key_dim)`  
* `W_v`: `(d_model_q, val_dim)`  

입력 시퀀스를 `X`들이라고 할 때, query, key, value의 계산 과정과 shape들은 다음과 같다.

* `Q = X_q @ W_q`    # shape: `(batch_size, seq_len_q, key_dim)`
* `K = X_k @ W_k`    # shape: `(batch_size, seq_len_k, key_dim)`
* `V = X_v @ W_v`    # shape: `(batch_size, seq_len_k, val_dim)`

### 차원 관계 정리

| 입력 `X`          | `W`                                | 쿼리, 키, 값 |
| ------------- | --------------------------------- |-----------------|
| `X_q` : `(batch_size, seq_len_q, d_model_q)` | `W_q` : `(d_model_q, key_dim)` | `Q` :  `(batch_size, seq_len_q, key_dim)`
| `X_k` : `(batch_size, seq_len_k, d_model_k)` | `W_k`: `(d_model_k, key_dim)` | `K` : `(batch_size, seq_len_k, key_dim)`
| `X_v` : `(batch_size, seq_len_k, d_model_v)` | `W_v`: `(d_model_q, val_dim)` | `V` : `(batch_size, seq_len_k, val_dim)`


* `key_dim == query_dim`이어야 $\mathrm{Q} \, \mathrm{K}^{\top}$가 정의됨.
  
* `seq_len_k`와 `seq_len_q`는 시퀀스의 길이이며 같을 수도 있고, 다를 수도 있음. (ex: 인코더는 10, 디코더는 5)
  
* key의 dimension(`key_dim`)과 value의 dimension(`val_dim`)은 같을 수도 있고, 다를 수도 있음

* `d_model_q`, `d_model_k`, `d_model_v` 또한 모두 같을 수도 있고 다를 수도 있음

### Attention 연산 요약

1. 유사도 계산 (Dot Product)
   - $\text{scores} = \mathrm{Q} \, \mathrm{K}^{\top}, \quad$ shape:  `(batch_size, seq_len_q, seq_len_k)` <br><br>

2. Scaling (정규화)
   - $\text{scaled\_scores} = \frac{\mathrm{Q} \, \mathrm{K}^{\top}}{\sqrt{\text{key\_dim}}}$ <br><br>

3. Softmax로 확률 분포화 (각 query 시점 마다)
   - $\text{Attention Weights} = \text{softmax}(\text{scaled\_scores})$ <br><br>

4. Weighted sum of values 
   - $\text{context} = \text{Attention Weights} \cdot \mathrm{V}, \quad $  shape: `(batch_size, seq_len_q, val_dim)`

#### Scaling을 하는 이유

- Dot product attention은 종종 `key_dim`(depth)의 제곱근으로 조정되는데, 이는 model의 depth가 클 경우 dot product의 값이 매우 커질 수 있기 때문이다. 

- 이렇게 되면 softmax function이 매우 좁은 범위에서 작동하게 되어, gradient가 작은 부분에서 매우 급격한 변화가 발생하는 'sharp' softmax가 생성될 수 있다.

- 이러한 문제를 피하기 위해 depth의 제곱근으로 dot product 값을 나누어 조정한다.

아래 예제에서 `Q`, `K`, `V`는 `W`들과의 연산을 통해 이미 만들어진 query, key, value의 계산 결과라고 가정하자.

In [6]:
np.set_printoptions(suppress=True)

Q = np.array([[0, 0, 10],
                   [0, 10, 0],
                   [10, 10, 0]])  # (seq_len_q, key_dim) = (3, 3)

K = np.array([[10, 0, 0],
                   [0, 10, 0],
                   [0, 0, 10],
                   [0, 0, 10]])  # (seq_len_k, key_dim) = (4, 3)

V = np.array([[1, 0],
                   [10, 0],
                   [100, 5],
                   [1000, 6]])  # (seq_len_k, val_dim) = (4, 2)

key_dim = np.shape(K)[-1] # 3
query_dim = np.shape(Q)[-1] # 3

seq_len_q = np.shape(Q)[-2]  # 3
seq_len_k = np.shape(K)[-2]  # 4
seq_len_v = np.shape(V)[-2]  # 4


matmul_qk = Q @ K.T   # attention score, (seq_len_q, seq_len_k) = (3, 4)

scaled_attention_logits = matmul_qk / math.sqrt(key_dim) # scaled attention score

attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (seq_len_q, seq_len_k) = (3, 4)

output = tf.matmul(attention_weights, V)  # (seq_len_q, depth_v) = (3, 2)

아래 코드에서 `attention_weights`을 프린트한 결과를 보면, 주어진 query가 value의 어디에 집중해야 하는지 설명한다.

- Query `temp_q`의 첫번째 값은 value의 세번째와 네번째 값에 집중해야 한다.
- Query `temp_q`의 두번째 값은 value의 두번째 값에 집중해야 한다.
- Query `temp_q`의 세번째 값은 value의 첫번째와 두번째 값에 집중해야 한다.

In [7]:
print("Attention weight : \n", attention_weights)

Attention weight : 
 tf.Tensor(
[[0.  0.  0.5 0.5]
 [0.  1.  0.  0. ]
 [0.5 0.5 0.  0. ]], shape=(3, 4), dtype=float64)


Attention weight를 value에 적용된 결과는 다음과 같다.

- Query `Q`의 첫번째 값의 의미는 `[550.    5.5]`으로 해석됨.
- Query `Q`의 두번째 값의 의미는 `[ 10.    0. ]`으로 해석됨.
- Query `Q`의 세번째 값의 의미는 `[  5.5   0. ]`으로 해석됨.

Output의 sequence 길이는 `seq_len_q`와 같고, dimension은 `dim_v`와 같다.

In [8]:
print("Output  : \n", output)

Output  : 
 tf.Tensor(
[[550.    5.5]
 [ 10.    0. ]
 [  5.5   0. ]], shape=(3, 2), dtype=float64)


## Multi-head attention

**Multi-Head Attention**에서는 

* 쿼리 Q, 키 K, 값 V의 차원을 `num_heads`개의 subspace로 분할하고, 각 head마다 독립적인 attention을 수행한 뒤,

* 그 결과들을 **결합(concatenate)** 후 다시 선형 변환하여 하나의 출력으로 만든다.

각, head별 쿼리와 키의 차원을 `key_dim`이라 하자.

그러면 `key_dim * num_head`는 `Q` 전체의 feature 차원의 개수가 된다. 

예를 들어, `Q` 전체의 feature 차원이 `512`, `num_heads = 8`이면, 각 head에서 키와 쿼리는 `512 / 8 = 64`차원짜리 feature를 갖게 되는 셈.

In [9]:
Q = np.array([[0, 0, 10, 10],
                   [0, 10, 0, 0],
                   [10, 10, 0, 0]])  # (4, 3)

K = np.array([[10, 0, 0, 0],
                   [0, 10, 0, 10],
                   [0, 0, 10, 0],
                   [0, 0, 10, 0]])  # (4, 4)

V = np.array([[1, 0],
                   [10, 0],
                   [100, 5],
                   [1000, 6]])  # (4, 2)


total_key_dim = np.shape(K)[-1] # 4
total_v_dim = np.shape(V)[-1] # 2
num_heads = 2

key_dim = total_key_dim// num_heads
v_dim = total_v_dim // num_heads

Q = np.reshape(Q, (-1, num_heads, key_dim)) # shape = (seq_len, num_heads, key_dim)
Q = Q.transpose((1, 0, 2)) # shape = (num_heads, seq_len, key_dim)

K = np.reshape(K, (-1, num_heads, key_dim))
K = K.transpose((1, 0, 2))

V = np.reshape(V, (-1, num_heads, v_dim))
V = V.transpose((1, 0, 2))

matmul_qk = np.matmul(Q, K.transpose((0, 2, 1)))   # attention score

scaled_attention_logits = matmul_qk / math.sqrt(key_dim) # scaled attention score, (..., seq_len_q, seq_len_k)

attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

print(attention_weights)  # num_heads 개의 attention_weights들

tf.Tensor(
[[[0.25       0.25       0.25       0.25      ]
  [0.         1.         0.         0.        ]
  [0.5        0.5        0.         0.        ]]

 [[0.         0.33333333 0.33333333 0.33333333]
  [0.25       0.25       0.25       0.25      ]
  [0.25       0.25       0.25       0.25      ]]], shape=(2, 3, 4), dtype=float64)


최종 output, 즉, 쿼리 별 value 결과값은 이전과 마찬가지로 `(seq_len_q, dim_v)`의 shape을 가진다.

In [10]:
output = tf.matmul(attention_weights, V)  
print(output)

tf.Tensor(
[[[277.75      ]
  [ 10.        ]
  [  5.5       ]]

 [[  3.66666667]
  [  2.75      ]
  [  2.75      ]]], shape=(2, 3, 1), dtype=float64)


In [11]:
output = np.transpose(output, (1, 0, 2))  # (seq_len_q, num_heads, v_dim)
output = np.reshape(output, (-1, total_v_dim))  # (seq_len_q, total_v_dim)

print(output)

[[277.75         3.66666667]
 [ 10.           2.75      ]
 [  5.5          2.75      ]]


## Multi-Head Self Attention

Transformer의 Multi-Head Self Attention의 작동 방식은 다음과 같다.

- 이전에 공부한 어텐션 메커니즘에서는 query가 디코더에서 제공되고, key와 value는 인코더에서 제공된다. 이는 디코더가 인코더의 출력과 상호작용하여 관련 정보를 추출하는 방식.

- **셀프** 어텐션에서는 query, key, value를 모두 동일한 입력 시퀀스에서 생성한다. 즉, 인코더 입력 데이터 자체에서 query, key, value를 추출함.

- 이 방식은 입력 시퀀스의 각 요소가 시퀀스 내의 다른 모든 요소와 상호작용하여, 자신의 문맥(context)을 이해하도록 도와준다.

- **Multi-Head** attention은 이러한 셀프 어텐션을 여러 헤드로 병렬 처리하여, 서로 다른 서브스페이스의 정보를 학습할 수 있게 한다. 각 헤드는 입력 데이터의 다른 부분에 초점을 맞추어 보다 풍부한 표현을 학습하는 효과가 있다.

- 이를 통해 모델은 시퀀스 내의 긴 의존성을 효과적으로 캡처할 수 있으며, 자연어 처리와 같은 분야에서 문맥을 이해하는 데 매우 유용하다.

- Self-attentions에서는 자연스럽게 `X_q = X_k`, `Q = K`이 된다. 단, value는 key와 다르게 지정 가능

케라스에서는 [`tensorflow.keras.MultiHeadAttention`](https://keras.io/api/layers/attention_layers/multi_head_attention/)를 이용하여 구현할 수 있다.

### MultiHeadAttention 구조

`MultiHeadAttention`에서의 작업 흐름은 다음과 같다.

```
Input Tensor → [Dense Layer for Query (W_q)] → Queries (Q)
Input Tensor → [Dense Layer for Key (W_k)]   → Keys (K)
Input Tensor → [Dense Layer for Value (W_v)] → Values (V)

[Split into h heads]  (Q → Q₁...Q_h,  K → K₁...K_h,  V → V₁...V_h)

[For each attention head]                                                                      
     [Scaled Dot Product Attention] (Q_i · K_i^T / sqrt(depth)) → Attention Scores
        → [Softmax] → Attention Weights
        → [Dot Product] (Attention Weights, V_i)
        → Attention Outputs (for each head)
               ↓                       
[Concatenate Heads] → Combined all Head Outputs → [Dense Layer] →  Output

```

Self multi head attention 결과는 입력 시퀀스의 복잡한 관계를 학습하여, 시퀀스 내의 문맥 정보를 풍부하게 표현한 결과라고 볼 수 있다.

`MultiHeadAttention`의 주요 인자는 다음과 같다.

- `num_heads` : 병렬 처리되는 head들의 수.  head는 input dimension을 나누어서 사용하기 때문에, 일반적으로 input dimension이 head의 수로 나누어떨어지도록 설정된다.
  
- `key_dim` : 각 head 별 query 및 key의 벡터 차원, 전체 query 및 key 벡터는 `num_heads * key_dim`의 차원을 가짐

`MultiHeadAttention`의 call argument로서 `query`, `key`, `value`가 존재하는데, 이들은 각각 Q, K, V를 생성하는데 사용되는 입력 벡터 역할을 한다고 생각하면 된다.

- 즉, $W$들이 곱해지기 전의 값.

- 마지막 Dense layer는 Concatenate된 head의 결과를 모델의 기대하는 차원으로 투영하는 역할. Timedistrbuted로 처리 됨.

  - `output_shape`을 따로 지정할 수도 있고, default로 `output_shape = None`일 경우 (쿼리) 입력 차원과 동일

### `MultiHeadAttention` 파라미터 분석

간단하게 `key_dim = value_dim`이고, `output_shape`을 따로 설정하지 않은 상황, 즉, `output_dim = input_dim`에서 살펴보자.  

- Q, K, V의 weight matrix들

  - Q, K, V는 기본적으로 `input_dim` 차원의 벡터를 `num_heads * key_dim` 차원으로 보내는 역할
  
  - 각각 `(input_dim, num_heads * key_dim)` 크기의 행렬, 즉 각각 `input_dim * num_heads * key_dim`의 파라미터 수를 가짐.

- Bias term for Q, K, V: 각각 `num_heads * key_dim` 개수의 bias term이 있음

- Output projection dense layer 가중치 :

  - Head output을 모두 concat한 후, 최종 출력 차원으로 다시 투영하는 dense layer : `(num_heads * value_dim) → output_dim`
  
    - 단, 아래 예제에서는 `key_dim = value_dim`을 가정 중  <br><br>
  - W: shape = (`num_heads * key_dim`, `output_dim`), 보통 `output_dim = input_dim`

  - bias : `output_dim`개

In [12]:
# 이 값들을 변경해 가며 테스트해 보자.
input_dim = 24
key_dim = 3
num_heads = 8

encoder_inputs = keras.Input(shape=(timestep, input_dim))
MHA_layer = layers.MultiHeadAttention(key_dim=key_dim, num_heads=num_heads)
x = MHA_layer(encoder_inputs, encoder_inputs) # (query, value). key는 생략, 즉, key=query로 자동 설정
model = keras.Model(encoder_inputs, x)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 6, 24)]      0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 6, 24)       2400        ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
Total params: 2,400
Trainable params: 2,400
Non-trainable params: 0
__________________________________________________________________________________________________


In [13]:
# Q, K, V 계산을 위한 W와 bias들 파라미터의 수
3 * (input_dim * num_heads * key_dim + num_heads * key_dim)

1800

In [14]:
# 마지막 dense layer의 파라미터 수. output_dim = input_dim
num_heads * key_dim * input_dim +  input_dim

600

In [15]:
# 총합
3 * (input_dim * num_heads * key_dim + num_heads * key_dim) + num_heads * key_dim * input_dim +  input_dim

2400

물론 `key_dim != value_dim`과 `input_dim != output_dim`인 경우도 파라미터 개수를 계산해 볼 수 있다.

In [16]:
# 이 값들을 변경해 가며 테스트해 보자.
timestep = 6
dim = 4

input_dim = 24
output_shape = 32  # output_dim
key_dim = 3
value_dim = 4
num_heads = 8

encoder_inputs = keras.Input(shape=(timestep, input_dim))
MHA_layer = layers.MultiHeadAttention(key_dim=key_dim, value_dim=value_dim, num_heads=num_heads, output_shape=output_shape)
x = MHA_layer(encoder_inputs, encoder_inputs)  # (query, value). key는 생략, 즉, key=query로 자동 설정
model = keras.Model(encoder_inputs, x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 6, 24)]      0           []                               
                                                                                                  
 multi_head_attention_1 (MultiH  (None, 6, 32)       3056        ['input_2[0][0]',                
 eadAttention)                                                    'input_2[0][0]']                
                                                                                                  
Total params: 3,056
Trainable params: 3,056
Non-trainable params: 0
__________________________________________________________________________________________________


In [17]:
# 파라미터 수 계산
# W_q, W_k, W_v와 bias들
pa = 2 * (input_dim * num_heads * key_dim + num_heads * key_dim) + (input_dim * num_heads * value_dim + num_heads * value_dim) 
# dense layer
pd = num_heads * value_dim * output_shape +  output_shape
pa + pd

3056

### 간단 예제

입력 시퀀스의 각 time step 벡터를 self-attention으로 처리하여, 그 결과를 그대로 예측하는 간단한 예제를 살펴보자.

In [18]:
# 하이퍼파라미터
seq_len = 5
d_model = 12
num_heads = 4
key_dim = d_model // num_heads
batch_size = 32

# 입력 데이터: 랜덤 시퀀스
X = np.random.randn(1000, seq_len, d_model).astype(np.float32)
Y = X.copy()  # 목표는 그대로 복사

# 모델 구성
inputs = tf.keras.Input(shape=(seq_len, d_model))

# MHA Layer
attention_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(inputs, inputs)

# Optional: Dense layer (identity map)
outputs = layers.Dense(d_model)(attention_out)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="adam", loss="mse")
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 5, 12)]      0           []                               
                                                                                                  
 multi_head_attention_2 (MultiH  (None, 5, 12)       624         ['input_3[0][0]',                
 eadAttention)                                                    'input_3[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 5, 12)        156         ['multi_head_attention_2[0][0]'] 
                                                                                                  
Total params: 780
Trainable params: 780
Non-trainable params: 0
____________________________

In [19]:
# 학습
model.fit(X, Y, epochs=100, batch_size=batch_size, validation_split=0.1)

Epoch 1/100
29/29 [==============================] - 0s 5ms/step - loss: 0.9979 - val_loss: 0.9633
Epoch 2/100
29/29 [==============================] - 0s 3ms/step - loss: 0.9468 - val_loss: 0.9163
Epoch 3/100
29/29 [==============================] - 0s 3ms/step - loss: 0.8920 - val_loss: 0.8597
Epoch 4/100
29/29 [==============================] - 0s 3ms/step - loss: 0.8263 - val_loss: 0.7928
Epoch 5/100
29/29 [==============================] - 0s 3ms/step - loss: 0.7511 - val_loss: 0.7140
Epoch 6/100
29/29 [==============================] - 0s 3ms/step - loss: 0.6683 - val_loss: 0.6266
Epoch 7/100
29/29 [==============================] - 0s 3ms/step - loss: 0.5835 - val_loss: 0.5429
Epoch 8/100
29/29 [==============================] - 0s 3ms/step - loss: 0.5087 - val_loss: 0.4785
Epoch 9/100
29/29 [==============================] - 0s 3ms/step - loss: 0.4524 - val_loss: 0.4305
Epoch 10/100
29/29 [==============================] - 0s 3ms/step - loss: 0.4042 - val_loss: 0.3859
Epoch 11/

In [20]:
test_input = np.random.randn(1, seq_len, d_model).astype(np.float32)
pred = model.predict(test_input)

print("Input:")
print(np.round(test_input[0], 2))

print("Prediction:")
print(np.round(pred[0], 2))

1/1 [==============================] - 0s 57ms/step
Input:
[[ 0.83  1.31 -1.45 -0.65 -0.67 -0.12  0.92 -1.6  -1.03  2.05  0.31 -0.36]
 [-1.69  0.99  0.39  1.42  0.05  1.58 -0.74 -0.28 -2.58  0.98  0.3  -1.03]
 [ 0.54 -0.36  1.66  0.4  -0.24  0.41  0.2  -0.66  1.31  0.81 -0.56  0.74]
 [-1.07 -0.43 -1.47 -0.05 -0.11 -1.06 -1.71 -0.65  0.8   0.61 -0.11 -1.04]
 [-0.54 -0.95 -0.66  1.58 -0.17 -0.7  -0.07 -0.09 -1.74  0.38 -0.07 -1.26]]
Prediction:
[[ 0.25  1.6  -1.82  0.05 -0.73  0.35  0.05 -1.66 -1.11  2.46  0.65 -0.93]
 [-1.74  0.37  0.51  1.12 -0.44  1.17 -0.42 -0.53 -2.52  1.02  0.16 -1.63]
 [ 0.05 -0.71  1.24  0.46  0.14  0.71  0.96 -0.54  1.05  1.3  -0.15  0.8 ]
 [-1.27 -0.09 -1.09 -0.05 -0.46 -0.88 -2.1  -0.51 -0.13  1.11  0.96 -1.26]
 [-0.83 -0.05 -0.5   1.1  -0.5  -0.23 -0.27 -0.43 -2.26  0.67  0.36 -1.65]]


## 인코더

인코더 모델을 구성하는 함수를 정의하자. 

인코더 모델은 self-attetion 부분과 feed-forward 부분으로 구성된다.

- 앞에서 공부한 인코더-디코더 모델에서 순환 신경망 대신 self-attention이 있는 형태

Feed-forward 부분은 self-attention 메커니즘 이후에 입력 데이터에 대한 추가적인 비선형 변환을 수행하는 단계이다.

이는 모델이 더 복잡하고 다양한 패턴을 학습할 수 있도록 도와준다. 

Transformer encoder의 feed-forward 부분은 일반적으로 두 개의 Dense 레이어와 활성화 함수로 구성된다. 


### 인코더의 주요 구성

Transformer의 인코더는 하나의 블록 안에 다음과 같은 주요 구성요소를 갖는다.

1. Multi-Head Self-Attention Layer

   * Transformer는 문장을 처리하기 위해 개발된 것으로, self-attention은 입력 시퀀스 내에서 단어들 간의 상호관계를 학습
   * 같은 시퀀스 안에서 query, key, value가 동일하기 때문에 self-attention이라고 부름 <br><br>

2. Layer Normalization
   * 각 샘플의 각 time step 별로 해당 feature 차원(`axis=-1`)에 대해 평균과 분산을 계산하여 정규화하는 과정 → 학습 안정화, 수렴 향상
   * Transformer에서는 Residual 연결 이후 또는 이전에 항상 LayerNormalization을 적용 <br><br>

3. Residual connection : attention ouput + input
   * Attention의 결과에 original 입력을 더해주는 과정을 residual connection이라고 부른다.
   * `inputs`까지 직접적인 경로(identity path)를 만들어서, 역전파 시 gradient가 입력까지 잘 전달되게 함
   * 완전히 새로 생성된 출력을 쓰기보단 기존 정보 + 조정된 정보로 자연스럽게 업데이트 <br><br>

4. Position-wise Feed-Forward Network (FFN)

   * 각 위치의 출력 벡터를 독립적으로 변환
   * 일반적으로 `Dense → ReLU → Dropout → Dense` 
   * 보통 첫 번째 dense layer에서 차원이 확장되고, 두 번째 dense layer에서 원래 차원으로 복원 <br><br>

5. 마지막으로 Residual connection 한 번 더 수행
   * 역전파 시 Gradient 흐름을 원활하게 함
   * 초기엔 FFN이 거의 학습되지 않아도 residual은 그대로 전달하여 정보 손실 방지

아래 인코더 모델의 텐서 흐름은 다음과 같다.

```
Input → [Layer Normalization] → [Multi-Head Attention] → [Dropout] → self-attention result
    ↓      
→ [Sum](Input + self-attention result) → <Residual>

→ [Layer Normalization] → [Dense (ReLU)] → [Dropout] → [Dense] → Feedforward result  
                                          
→ [Sum](<Residual> + Feedforward result) → Encoder output
```


In [21]:
# 인코더 모델 구성
def transformer_encoder(inputs, num_heads, key_dim, ff_dim, dropout=0.1):
    # === Self-Attention Block ===
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    MHA_layer = layers.MultiHeadAttention(key_dim=key_dim, num_heads=num_heads, dropout=dropout)
    
    x = MHA_layer(query=x, value=x, key=x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

     # === Feed-Forward Block ===
    ff = layers.LayerNormalization(epsilon=1e-6)(res)
    ff = layers.Dense(units=ff_dim, activation="relu")(ff)  # ff_dim으로 확장
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(units=inputs.shape[-1])(ff) # 원래 차원으로 복원
    return ff + res

적절한 input tensor를 생성하여 `transformer_encoder`에 입력하면 output tensor를 얻게 되어 추후에 모델을 생성할 때 이용할 수 있다.

예를 들어, 다음과 같은 코드를 보라.

In [22]:
encoder_inputs = keras.Input(shape=(timestep, dim))

encoder_ouputs = transformer_encoder(encoder_inputs, num_heads = 2, key_dim = 8, ff_dim = 10, dropout = 0.1)
encoder_ouputs

<KerasTensor: shape=(None, 6, 4) dtype=float32 (created by layer 'tf.__operators__.add_1')>

## 디코더

디코더는 인코더와 비슷하나, 어텐션이 다음 세 단계에 걸쳐 이루어 진다.

- **Masked self attention**
  - 디코더가 순차적으로 값을 생성할 때 현재까지 생성된 값들만을 이용하여 attention을 수행한다.
  - 즉, 미래에 디코더가 생성할 값은 attention 계산에 사용하지 않겠다는 의미이다.<br><br>
    
- Encoder-Decoder attention
  - 디코더 출력과 인코더 출력 간의 attention   
  - 디코더가 특정 시점 값을 생성할 때 인코더 출력 전체를 살펴보므로 masked attention은 아니다. <br><br>
 
- Position-wise Feedforward Network

### Mask 행렬

마스크 행렬은 어디를 가릴지 위치를 나태내 주는 행렬.

- 가려야 할 위치가 1이고, 나머지는 0.
- 가장 기본적으로 upper triangular part가 1이고 나머지는 0인 형태가 새용됨

Transformer에서 attention mask는 `softmax`에 들어가기 전 score 행렬 `QKᵀ`에 다음과 같이 적용된다.

$$
\text{Attention}(\mathbf{Q}, \mathbf K, \mathbf V) = \text{softmax} \left( \frac{\mathbf Q \mathbf{K}^\top}{\sqrt{d_k}} +  \text{mask} \cdot(-\infty )\right) \mathbf{V}
$$

- 즉, softmax를 적용했을 때, 0이 되게 하기 위함
  
- 실제로는 $-\infty$를 곱할 수 없기 때문에, `-1e9`와 같은 매우 작은 수를 곱함

Mask 행렬을 위해 다음 함수를 정의하자.

여기서 `size`는 `timestep`에 해당 됨.

In [23]:
def create_look_ahead_mask(size):
    # band_part(A, -1, 0)은 lower triangular 행렬을 추출
    return 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
create_look_ahead_mask(timestep)

<tf.Tensor: shape=(6, 6), dtype=float32, numpy=
array([[0., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1.],
       [0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)>

활용 시에는 아래와 같이 인자로 전달한다.

In [24]:
temp = np.random.randn(1, 3, 4)
x = np.concatenate([temp, temp], axis = 1)

layers.MultiHeadAttention(num_heads=1, key_dim=1)(query=x, value=x, attention_mask=create_look_ahead_mask(timestep))

<tf.Tensor: shape=(1, 6, 4), dtype=float32, numpy=
array([[[-0.10585935, -0.28205392,  0.07306184,  0.33768246],
        [-0.00079814, -0.00212657,  0.00055086,  0.00254598],
        [-0.07221033, -0.19239873,  0.04983801,  0.23034488],
        [ 0.02255827,  0.06010474, -0.01556923, -0.07195899],
        [ 0.06998947,  0.18648142, -0.04830521, -0.2232605 ],
        [-0.07444675, -0.19835751,  0.05138154,  0.23747888]]],
      dtype=float32)>

아래 디코더 모델의 텐서 흐름은 다음과 같다.

```
Decoder Input  → [Layer Normalization] → [Masked Self Multi-Head Attention] → [Dropout] → self-attention result
    ↓
→ [Sum](Decoder Input + self-attention result) → <Residual1>
                                                                 
Encoder Output → [Encoder-Decoder Multi-Head Attention with Encoder Output] → [Dropout] → Encoder-Decoder attention result

→ [Sum](<Residual1> + Encoder-Decoder attention result) → <Residual2>
        
→ [Layer Normalization] → [Dense (ReLU)] → [Dropout] → [Dense] → Feedforward result  
                                          
→ [Sum](<Residual2> + Feedforward result) → Decoder output
```

In [25]:
# Transformer 디코더 블록
def transformer_decoder(inputs, encoder_output, num_heads, key_dim, ff_dim, dropout=0.1):
    seq_len = tf.shape(inputs)[1]
    look_ahead_mask = create_look_ahead_mask(seq_len)

    # Masked Multi-Head Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    MHA_layer1 = layers.MultiHeadAttention(key_dim=key_dim, num_heads=num_heads, dropout=dropout)
    x = MHA_layer1(query=x, value=x, key=x, attention_mask=look_ahead_mask)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Encoder-Decoder Attention
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    MHA_layer2 = layers.MultiHeadAttention(key_dim=key_dim, num_heads=num_heads, dropout=dropout)
    x = MHA_layer2(query=x, value=encoder_output, key=encoder_output)
    x = layers.Dropout(dropout)(x)
    res = x + res

    # Feed Forward Part
    ff = layers.LayerNormalization(epsilon=1e-6)(res)
    ff = layers.Dense(units=ff_dim, activation="relu")(ff)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(units=inputs.shape[-1])(ff)
    return x + res

## Transformer 모델 구성

위에서 정의된 인코더 디코더를 이용하여 transformer 모형을 구성해 보겠다.

`build_transformer_model` 함수 내에서 반복문을 이용하여 encoder와 decoder를 여러 블록으로 설정할 수 있다.

- 각 블록은 입력 데이터에 대한 더 복잡한 패턴과 관계를 학습할 수 있도록 도와주고,
- 깊은 구조는 멀리 떨어진 토큰 간의 복잡한 관계도 포착한다고 알려져 있다.

전체 모델 구조는 다음과 같이 간단히 나타낼 수 있다. 
    
```
[Encoder Input] ─► PositionalEncoding ─► [Encoder Blocks] ─► encoder_outputs
                                                                  ↓
[Decoder Input] ─► PositionalEncoding ─► [Decoder Blocks using encoder_outputs] ─► Final Dense Layer
```

In [26]:
def build_transformer_model(input_shape, key_dim, num_heads, ff_dim, num_encoder_blocks, num_decoder_blocks, dropout=0.1):
    encoder_inputs = keras.Input(shape=input_shape)
    x = PositionalEncoding(input_shape[0], input_shape[1])(encoder_inputs)
    for _ in range(num_encoder_blocks):
        x = transformer_encoder(x, key_dim, num_heads, ff_dim, dropout)
    encoder_outputs = x

    #decoder_inputs = keras.Input(shape=(input_shape[0] + 1, input_shape[1]))
    decoder_inputs = keras.Input(shape=(None, input_shape[1]))
    x = PositionalEncoding(input_shape[0] + 1, input_shape[1])(decoder_inputs)
    for _ in range(num_decoder_blocks):
        x = transformer_decoder(x, encoder_outputs, key_dim, num_heads, ff_dim, dropout)

    outputs = layers.Dense(input_shape[1], activation="linear")(x)
    return keras.Model([encoder_inputs, decoder_inputs], outputs)

간단한 트랜스포머 모델을 구현하여 보자.

In [27]:
# 모델 하이퍼파라미터
timestep = 6
dim = 12
num_heads = 4
key_dim = dim // num_heads

ff_dim = 64
num_encoder_blocks = 2
num_decoder_blocks = 2
dropout = 0.1

input_shape = (timestep, dim)
model = build_transformer_model(input_shape, key_dim, num_heads, ff_dim, num_encoder_blocks, num_decoder_blocks, dropout)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 6, 12)]      0           []                               
                                                                                                  
 positional_encoding_1 (Positio  (None, 6, 12)       0           ['input_5[0][0]']                
 nalEncoding)                                                                                     
                                                                                                  
 layer_normalization_2 (LayerNo  (None, 6, 12)       24          ['positional_encoding_1[0][0]']  
 rmalization)                                                                                     
                                                                                            

In [28]:
model.compile(optimizer='adam', loss='mean_squared_error')

### 훈련 데이터

`input_sequences`: `(5000, timestep, dim)` 크기의 랜덤 시퀀스 데이터를 생성하여 학습용 인코더 입력으로 사용해 보자.

또한 디코더 시작 토큰(`start token`) 역할을 하는 0 벡터를 이용하여 `decoder_input_sequences`를 생성한다.

* 디코더 입력 = 시작 토큰 + 인코더 입력 시퀀스  
* 타깃 시퀀스 = 인코더 입력 시퀀스 + 끝 토큰  
* 즉, 입력을 한 칸 오른쪽으로 shift한 형태
  .

In [29]:
# 입력 데이터: 랜덤 시퀀스
num_samples = 5000
input_sequences = np.random.randn(num_samples, timestep, dim).astype(np.float32)
token = np.zeros((num_samples, 1, dim), dtype=np.float32)
decoder_input_sequences = np.concatenate([token, input_sequences], axis=1)
target_sequences = np.concatenate([input_sequences, token], axis=1)


### 훈련 과정

* 인코더 입력 = `input_sequences`
* 디코더 입력 = `decoder_input_sequences`
* 출력(타깃): `target_sequences`
  
모델이 다음 시점 토큰을 예측하도록 학습한다.

Transformer seq2seq 모델을 자기회귀(auto-regressive) 학습 형태로 훈련하는 예제이며, 코더는 이전까지의 토큰(start token 포함)을 입력받아 다음 토큰을 맞추도록 학습한다.

이 Transformer 모형은 시계열 예측 문제를 잘 학습하는 편은 아니다.

In [30]:

# 학습
model.fit(
    [input_sequences, decoder_input_sequences],
    target_sequences,
    epochs=100,
    batch_size=32
)

Epoch 1/100
157/157 [==============================] - 3s 12ms/step - loss: 1.2731
Epoch 2/100
157/157 [==============================] - 2s 12ms/step - loss: 0.8722
Epoch 3/100
157/157 [==============================] - 2s 11ms/step - loss: 0.7736
Epoch 4/100
157/157 [==============================] - 2s 11ms/step - loss: 0.7150
Epoch 5/100
157/157 [==============================] - 2s 10ms/step - loss: 0.6711
Epoch 6/100
157/157 [==============================] - 2s 11ms/step - loss: 0.6409
Epoch 7/100
157/157 [==============================] - 2s 11ms/step - loss: 0.6218
Epoch 8/100
157/157 [==============================] - 2s 12ms/step - loss: 0.6064
Epoch 9/100
157/157 [==============================] - 2s 12ms/step - loss: 0.5965
Epoch 10/100
157/157 [==============================] - 2s 12ms/step - loss: 0.5880
Epoch 11/100
157/157 [==============================] - 2s 12ms/step - loss: 0.5797
Epoch 12/100
157/157 [==============================] - 2s 11ms/step - loss: 0.5753
E

### 예측 과정

Autoregressive하게 예측하는 과정을 살펴보자.

* 인코더 입력(`test_input`)을 모델에 고정시켜 둠.
* 디코더 입력(`decoder_input`)은 처음에는 `[0]` 토큰 하나로 시작.
* 한 스텝씩 모델의 출력을 얻고, 해당 시점의 토큰을 디코더 입력에 추가.
* 이렇게 autoregressive(자기회귀) 방식으로 전체 시퀀스를 생성.

훈련이 잘 된 것이 아니라 시계열 예측이 잘 되고 있지는 않지만 일반적인 예측 과정을 공부하는 차원에서 살펴보자.

In [31]:
predicted = []
test_input = np.random.uniform(0, 100, size=(1, timestep, dim)).astype(np.float32)
decoder_input = np.zeros((1, 1, dim), dtype=np.float32)

for t in range(timestep):
    out = model.predict([test_input, decoder_input], verbose=0)
    next_token = out[:, t:t+1, :]  # 현재 t 위치의 출력
    predicted.append(next_token)
    decoder_input = np.concatenate([decoder_input, next_token], axis=1)

predicted_seq = np.concatenate(predicted, axis=1)  # [1, T, D]

In [32]:
# 결과 출력
print("=== Input Sequence ===")
print(np.round(test_input[0], 2))   # 원래 입력 시퀀스

print("\n=== Predicted Sequence ===")
print(np.round(predicted_seq[0], 2))  # 모델이 step-by-step으로 생성한 출력

=== Input Sequence ===
[[ 5.9  84.48 55.46 60.91 39.75 25.66 16.88 49.7  13.09 82.83 67.29 23.87]
 [86.38 84.48 41.02 54.33 39.05  2.61  2.98 88.1  59.53 55.66 72.29 91.86]
 [59.89 24.55 53.84 14.91 79.9  28.69 64.5  68.98 22.69 35.47 49.43  2.2 ]
 [82.73  7.92 74.   25.94 80.67 77.67 52.23 57.15 92.46 86.11 11.56 82.25]
 [95.52 74.13 31.54 23.44 96.42 39.99 29.29 37.63 14.87 42.31 96.79 14.39]
 [83.46 78.55 85.41 36.36 99.    4.62 73.88 71.37 34.87 70.11 34.02 65.76]]

=== Predicted Sequence ===
[[51.62 48.87 56.84 26.29 50.04 37.92 33.41 48.82 41.83 41.78 43.68 34.21]
 [48.66 50.31 48.82 41.38 34.74 36.19 35.28 42.52 48.47 42.86 47.28 41.06]
 [45.79 49.87 58.9  27.82 43.88 33.59 35.59 42.36 46.26 35.82 42.53 28.67]
 [42.78 48.1  48.96 38.   39.62 38.08 35.93 46.78 46.47 41.95 47.12 38.49]
 [52.76 52.65 55.94 32.61 49.91 43.06 47.63 34.98 44.82 46.53 42.54 46.07]
 [46.69 50.85 56.6  31.84 40.4  33.5  35.04 43.13 46.62 36.85 44.41 31.41]]
